### 天气api测试
https://home.openweathermap.org/api_keys




In [4]:
import requests
import json
 
def get_weather(city_name):
    api_key = ""  # 替换为你自己的API Key
    base_url = "http://api.openweathermap.org/data/2.5/weather"
 
    # 构建请求参数
    params = {
        "q": city_name if '市' in city_name else city_name+"市" , # 城市名
        "appid": api_key,       # API Key
        "units": "metric"       # 温度单位为摄氏度
    }
 
    # 发送GET请求获取天气数据
    response = requests.get(base_url, params=params)
    data = json.loads(response.text)
    # 解析返回的JSON数据
    if data["cod"] == 200:
        weather = data["weather"][0]["main"]           # 天气情况
        description = data["weather"][0]["description"] # 天气描述
        temperature = data["main"]["temp"]              # 温度
        humidity = data["main"]["humidity"]             # 湿度
        wind_speed = data["wind"]["speed"]              # 风速
 
    else:
        print("Failed to fetch weather data.")
    return data
get_weather("北京市")

Failed to fetch weather data.


{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

### 高德api测试

In [1]:
import pandas as pd
city_df = pd.read_excel('https://modelscope.oss-cn-beijing.aliyuncs.com/resource/agent/AMap_adcode_citycode.xlsx') # 读取成功需要在qwen-agent的环境下
city_df.head()

,中文名,adcode,citycode
0,中华人民共和国,0,NaN
1,北京市,110000,10.0
2,东城区,110101,10.0
3,西城区,110102,10.0
4,朝阳区,110105,10.0


In [2]:
token = ""
url = 'https://restapi.amap.com/v3/weather/weatherInfo?city={city}&key={key}'


In [31]:
def get_city_adcode(city_name):
        filtered_df = city_df[city_df['中文名'] == city_name]
        if len(filtered_df['adcode'].values) == 0:
            raise ValueError(f'location {city_name} not found, availables are {city_df["中文名"]}')
        else:
            return filtered_df['adcode'].values[0]
get_city_adcode("武汉市")

np.int64(420100)

In [30]:
get_city_adcode("朝阳区")

np.int64(110105)

In [18]:
url.format(city=get_city_adcode("朝阳区"),key=token)

'https://restapi.amap.com/v3/weather/weatherInfo?city=110105&key=812f1c50ed0cbc0fa79c5ddcf7e2b0ec'

In [32]:
import requests
# location=420100
response = requests.get(url.format(city=get_city_adcode("朝阳区"),key=token))

In [21]:
response.json()

{'status': '1',
 'count': '1',
 'info': 'OK',
 'infocode': '10000',
 'lives': [{'province': '北京',
   'city': '朝阳区',
   'adcode': '110105',
   'weather': '多云',
   'temperature': '29',
   'winddirection': '南',
   'windpower': '≤3',
   'humidity': '56',
   'reporttime': '2024-07-04 15:03:03',
   'temperature_float': '29.0',
   'humidity_float': '56.0'}]}

In [22]:
def call( params ) -> str:
        location = params['location']
        response = requests.get(url.format(city=get_city_adcode(location), key=token))
        data = response.json()
        if data['status'] == '0':
            raise RuntimeError(data)
        else:
            weather = data['lives'][0]['weather']
            temperature = data['lives'][0]['temperature']
            return f'{location}的天气是{weather}温度是{temperature}度。'

In [29]:
call({"location":"深圳市"})


'深圳市的天气是阴温度是26度。'